In [ ]:
import os

import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from scipy import sparse
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
rcParams['figure.figsize'] = (7,7)
import seaborn as sns
from scipy import sparse

import random
import tensorflow as tf

seed = 42


random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

palette = {'CNT':'#1f77b4',
 'DCT1':'#ff7f0e',
 'DCT2':'#279e68',
 'TL':'#279e68',
 'DCT':'#279e68',
 'ENDO':'#d62728',
 'FIB':'#aa40fc',
 'ICA':'#8c564b',
 'ICB':'#e377c2',
 'LEUK':'#b5bd61',
 'MES_FIB':'#17becf',
 'MES':'#17becf',
 'PC':'#aec7e8',
 'PEC':'#ffbb78',
 'PODO':'#98df8a',
 'PT':'#ff9896',
 'PT_VCAM1':'#c5b0d5',
 'TAL':'#c49c94',
 'Unclassified':'#808080',
 'Unknown':'#000000',
 'Low_Quality_RNA':'#808080'}


# In[3]:


import sys 
from deepscore import deepscore
from deepscore import marker_analysis
import pickle


import random
import tensorflow as tf

seed = 42


random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


In [ ]:

# LOAD the PROCESS the REFERENCE

# ## Preprocessing

ref_py = sc.read('../HORIZONTAL_RNA/objects/local.h5ad')  # LOAD the HCA Atlas
ref_py = ref_py.raw.to_adata()
ref_py.layers['counts'] = ref_py.X.copy()

ref_py.var['ENSG'] = ref_py.var.index.copy()
ref_py.var.index = ref_py.var['feature_name'].copy()

# SUBSET THE ATLAS TO MATCH OUR SAMPLE BIOLOGY


cortex_celltypes_l1= ['DCT',
 'PEC',
 'CNT',
 'POD',
 'PT',
 'IC',
 'IMM',
 'NEU',
 'VSM/P',
 'TAL',
 'EC',
 'FIB',
 'PC']

cortex_celltypes_l3= ['DCT1',
 'DCT2',
 'IC-B',
 'B',
 'MD',
 'CNT-IC-A',
 'MC',
 'CNT',
 'PT-S1/2',
 'CCD-IC-A',
 'PEC',
 'pDC',
 'EC-GC',
 'POD',
 'VSMC',
 'aPT',
 'CNT-PC',
 'FIB',
 'cDC',
 'REN',
 'EC-PTC',
 'T',
 'PT-S3',
 'ncMON',
 'NKC/T',
 'aTAL1',
 'C-TAL',
 'PL',
 'CCD-PC',
 'SC/NEU',
 'EC-LYM',
 'MDC',
 'N',
 'MAST',
 'aTAL2',
 'aFIB',
 'MYOF',
 'MAC-M2',
 'EC-AEA',
 'VSMC/P']

ref_py = ref_py[ref_py.obs['subclass.l1'].isin(cortex_celltypes_l1)]

ref_py = ref_py[ref_py.obs['subclass.l3'].isin(cortex_celltypes_l3)]

ref_py = ref_py[ref_py.obs['condition.long'].isin(['Normal Reference'])]


ref_py = ref_py[ref_py.obs['state.l2'].isin(['reference','adaptive - epi','adaptive - str'])].copy()


compute = False
cell_type = 'subclass.l3'

for i in ref_py.obs['subclass.l1'].unique().tolist():
    a = i.replace('/','_')
    # if os.path.exists(f'objects/reference_subset/{a}.h5ad') == False:
    ref_py[ref_py.obs['subclass.l1'].isin([i])].write(f'objects/reference_subset_clean/{a}.h5ad', compression='gzip')


# Get the L1 celltypes that have substates defined in HCA atlas L3

cell_type_list = ref_py.obs['subclass.l1'].unique().tolist()
single_cat_ct=[]
for i in ref_py.obs['subclass.l1'].unique().tolist():
    a = i.replace('/','_')
    ref_py = sc.read(f'objects/reference_subset_clean/{a}.h5ad')

    if len(ref_py.obs['subclass.l3'].unique().tolist()) == 1:
        print(a, 'Single category')
        single_cat_ct.append(i)

for i in single_cat_ct:
    cell_type_list.remove(i)
# cell_type_list = ['IMM', 'DCT', 'PT', 'CNT', 'TAL', 'IC', 'EC', 'FIB', 'VSM/P']


if compute:
    for i in cell_type_list:
        a = i.replace('/','_')
        ref_py = sc.read(f'objects/reference_subset_clean/{a}.h5ad')

        markers_filename= f'HCA_{a}_l3'
        print(a)

        sc.pp.normalize_total(ref_py, target_sum=1e4)
        sc.pp.log1p(ref_py)
    # Identify differentially expressed genes between cell types
        sc.tl.rank_genes_groups(ref_py, cell_type, method='wilcoxon', use_raw=False)
        ranked_genes_populations = ref_py.uns['rank_genes_groups'].copy()
        with open(f'markers_ds/{markers_filename}.pickle', 'wb') as handle:
            pickle.dump(ranked_genes_populations, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
# Generate Query SUBSETS to predict on
for mod in ['scRNA','snRNA','scRNA5p']:

    adata = sc.read(f'../HORIZONTAL_RNA/objects/{mod}_raw.h5ad', compression='gzip')
    adata.X = adata.layers['counts'].copy()
    # adata = adata[adata.obs['batch'].isin([mod])].copy()
    ds_hca = pd.read_csv(f'csv/Deepscore_HCA_l1_{mod}_CLEAN.csv', index_col=0)
    adata.obs['Deepscore_HCA_l1'] = ds_hca['Deepscore_HCA'].astype('category')
    adata.obs['Deepscore_HCA_l1_score'] = ds_hca['Deepscore_HCA_score'].astype('category')

    # In[33]:
    try:
        os.mkdir(f'objects/{mod}_subset/')
    except:
        print('Directory for subset saving already exists')


    for i in adata.obs['Deepscore_HCA_l1'].unique():
        print(i)
        a = i.replace('/','_')
        # if os.path.exists(f'objects/reference_subset/{a}.h5ad') == False:
        adata[adata.obs['Deepscore_HCA_l1'].isin([i])].write(f'objects/{mod}_subset/{a}.h5ad', compression='gzip')

Directory for subset saving already exists
IC
VSM/P
PT
IMM
PC
DCT
PEC
CNT
TAL
FIB
POD
EC
NEU
Directory for subset saving already exists
PT
DCT
IMM
VSM/P
FIB
EC
IC
PEC
TAL
PC
POD
CNT
NEU
Directory for subset saving already exists
IMM
DCT
EC
IC
PT
TAL
VSM/P
CNT
PC
POD
PEC
FIB
NEU


In [ ]:

n_markers = 300
cell_type = 'subclass.l3'

## SET PARAMETERS
for mod in ['scRNA','snRNA','scRNA5p']:
    # ## Train & Inference

    for i in cell_type_list:
        a = i.replace('/','_')
        if os.path.exists(f'csv/Deepscore_HCA_l3_{mod}_{a}_CLEAN.csv'):
            print(f'{mod} {a} already exists!')
        else:
            ref_py = sc.read(f'objects/reference_subset_clean/{a}.h5ad')
            adata = sc.read(f'objects/{mod}_subset/{a}.h5ad', compression='gzip')

            sc.pp.normalize_total(adata, target_sum=1e4)
            sc.pp.log1p(adata)

            markers_filename = f'HCA_{a}_l3'       
            # Identify differentially expressed genes between cell types

            with open(f'markers_ds/{markers_filename}.pickle', 'rb') as handle:
                ranked_genes_populations = pickle.load(handle) 

             # Step 1: Create a dictionary to store markers for each subset
            subset_markers_dict = {}

            # Step 2: Store markers for each subset
            for subset in ref_py.obs[cell_type].unique():
                subset_markers = ranked_genes_populations['names'][subset]
                subset_markers = [gene for gene in subset_markers if gene in adata.var.index]
                subset_markers_dict[subset] = set(subset_markers[:n_markers+100])

            # Step 3: Identify overlapping markers
            overlapping_markers = set()
            for subset, markers in subset_markers_dict.items():
                for other_subset, other_markers in subset_markers_dict.items():
                    if subset != other_subset:
                        overlapping_markers.update(markers.intersection(other_markers))

            # Step 4: Select markers for each subset, excluding overlapping markers
            marker_dict = {}
            for subset, markers in subset_markers_dict.items():
                unique_markers = [marker for marker in markers if marker not in overlapping_markers]
                marker_dict[subset] = unique_markers[:n_markers]  # Select up to 200 unique markers
                # print(subset,len(unique_markers[:n_markers]))
            selected_markers = [marker for subset in marker_dict for marker in marker_dict[subset]]

            # print(selected_markers)

            sc.pp.scale(ref_py)
            sc.pp.scale(adata) 


            # Subset the data to the selected markers
            ref_py = ref_py[:, list(selected_markers)].copy()
            adata = adata[:, list(selected_markers)].copy()

            ref_py.obs[cell_type] = ref_py.obs[cell_type].tolist()
            len(ref_py.obs[cell_type].unique())

            def scheduler(epoch, lr):
                if epoch < 10:
                    return lr
                else:
                    return lr * tf.math.exp(-0.1)


            n_feat = ref_py.shape[1]
            n_labs = len(ref_py.obs[cell_type].unique())

            ds = deepscore.DeepScore(hidden_nodes=[512, 256],
                           n_features=n_feat, 
                           n_labels=n_labs,
                           epochs=30,
                           batch_size=128, 
                           activation="relu", 
                           dropout=True, 
                           dropout_rate=0.3,
                           batchnorm=True, 
                           lr=0.001,
                           weight_reg=True)

            os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

            ds.set_reference(ref_py, label_by=cell_type, test_prop=0.1)

            ds.train(earlystopping=True, patience=10, lr_scheduler=scheduler,)

            prob_df, adata = ds.annotate(adata, pred_key='Deepscore_HCA',Unclassified = False,return_pred_matrix=True)

            adata.obs[['Deepscore_HCA','Deepscore_HCA_score']].to_csv(f'csv/Deepscore_HCA_l3_{mod}_{a}_CLEAN.csv')

            prob_df.to_csv(f'csv/prob_matrix/Deepscore_HCA_l3_{mod}_{a}_CLEAN.csv')



scRNA IMM already exists!


2024-01-29 12:11:47.716144: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-01-29 12:11:47.716187: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: IJC20724
2024-01-29 12:11:47.716195: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: IJC20724
2024-01-29 12:11:47.716507: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
2024-01-29 12:11:47.716912: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.125.6
2024-01-29 12:11:47.719038: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild Tenso

Model: "deepscore"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 1788)              7152      
_________________________________________________________________
dense512 (Dense)             (None, 512)               915968    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense256 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               10

2024-01-29 12:11:47.986818: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30
 33/293 [==>...........................] - ETA: 1s - loss: 2.0174 - categorical_accuracy: 0.3977

...

Model: "deepscore"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_78 (Batc (None, 1191)              4764      
_________________________________________________________________
dense512 (Dense)             (None, 512)               610304    
_________________________________________________________________
dropout_52 (Dropout)         (None, 512)               0         
_________________________________________________________________
batch_normalization_79 (Batc (None, 512)               2048      
_________________________________________________________________
dense256 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_53 (Dropout)         (None, 256)               0         
_________________________________________________________________
batch_normalization_80 (Batc (None, 256)               10

2024-01-29 12:21:57.754754: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2024-01-29 12:21:57.754780: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2024-01-29 12:21:57.754809: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


 30/113 [======>.......................] - ETA: 0s - loss: 1.0712 - categorical_accuracy: 0.5771

2024-01-29 12:21:58.368555: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2024-01-29 12:21:58.368573: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2024-01-29 12:21:58.408190: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2024-01-29 12:21:58.409322: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2024-01-29 12:21:58.411307: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./deepscore_logs/train/plugins/profile/2024_01_29_12_21_58

2024-01-29 12:21:58.412421: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ./deepscore_logs/train/plugins/profile/2024_01_29_12_21_58/IJC20724.trace.json.gz
2024-01-29 12:21:58.414155: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./deepscore_logs/train/plugins/profile/2024_01_29_12_21_5

113/113 [==============================] - 1s 6ms/step - loss: 0.6685 - categorical_accuracy: 0.7683 - val_loss: 0.2408 - val_categorical_accuracy: 0.9250
Epoch 2/30
113/113 [==============================] - 0s 4ms/step - loss: 0.3370 - categorical_accuracy: 0.8944 - val_loss: 0.2070 - val_categorical_accuracy: 0.9400
Epoch 3/30
113/113 [==============================] - 0s 4ms/step - loss: 0.2258 - categorical_accuracy: 0.9278 - val_loss: 0.1695 - val_categorical_accuracy: 0.9500
Epoch 4/30
113/113 [==============================] - 0s 4ms/step - loss: 0.2092 - categorical_accuracy: 0.9283 - val_loss: 0.2169 - val_categorical_accuracy: 0.9400
Epoch 5/30
113/113 [==============================] - 0s 4ms/step - loss: 0.1815 - categorical_accuracy: 0.9350 - val_loss: 0.2539 - val_categorical_accuracy: 0.9300
Epoch 6/30
113/113 [==============================] - 0s 4ms/step - loss: 0.1212 - categorical_accuracy: 0.9617 - val_loss: 0.2773 - val_categorical_accuracy: 0.9300
Epoch 7/30
113/